In [5]:
from flask import Flask, request, jsonify
from google.cloud import bigquery
import os
import pandas as pd
import json

In [6]:
project_id = "ticket-sauce-data"
try:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "ticket-sauce-data-58c586aceed9.json"
except:
    pass

In [7]:
app = Flask(__name__)
bq_client = bigquery.Client()

# Replace this with your staging table
STAGING_TABLE = "your_project.dataset.staging_tickets"

@app.route('/webhook', methods=['POST'])
def webhook():
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "Invalid JSON"}), 400

        # Handle both list of events or single event
        if isinstance(data, dict):
            records = [data]
        else:
            records = data

        df = pd.json_normalize(records)

        # Optional: Convert nested fields to string
        for col in df.columns:
            if df[col].apply(lambda x: isinstance(x, (dict, list))).any():
                df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (dict, list)) else x)

        # Upload to BigQuery
        job = bq_client.load_table_from_dataframe(df, STAGING_TABLE)
        job.result()

        return jsonify({"status": "success", "rows": len(df)}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500